In [2]:
import pandas as pd
import time
import numpy as np
import requests
import os
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import lightgbm as lgb
from xgboost import XGBRegressor
import seaborn as sns
import matplotlib.pyplot as plt
from catboost import CatBoostRegressor
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error
from sklearn.compose import TransformedTargetRegressor
from sklearn.preprocessing import FunctionTransformer
from tqdm import tqdm
pd.set_option('display.max_columns', None)  # Toutes les colonnes
pd.set_option('display.width', None)        # Pas de limite de largeur pour les lignes
pd.set_option('display.max_colwidth', None) # Pas de limite pour la largeur des cellules


# Préparation

In [3]:
import pandas as pd

def change_column_types(df):
    """
    Change les types de plusieurs colonnes dans un DataFrame pandas.

    Parameters:
        df (pd.DataFrame): Le DataFrame pandas.

    Returns:
        pd.DataFrame: Un nouveau DataFrame avec les colonnes modifiées.
    """
    # Dictionnaire des colonnes et leurs types cibles
    dict_type = {
        'id_mutation': 'string',
        'date_mutation': 'datetime64[ns]',
        'numero_disposition': 'string',
        'nature_mutation': 'string',
        'valeur_fonciere': 'float64',
        'adresse_numero': 'string',
        'adresse_suffixe': 'string',
        'adresse_nom_voie': 'string',
        'adresse_code_voie': 'string',
        'code_postal': 'string',
        'code_commune': 'string',
        'nom_commune': 'string',
        'code_departement': 'string',
        'ancien_code_commune': 'string',
        'ancien_nom_commune': 'string',
        'id_parcelle': 'string',
        'ancien_id_parcelle': 'string',
        'numero_volume': 'string',
        'lot1_numero': 'string',
        'lot1_surface_carrez': 'float64',
        'lot2_numero': 'string',
        'lot2_surface_carrez': 'float64',
        'lot3_numero': 'string',
        'lot3_surface_carrez': 'float64',
        'lot4_numero': 'string',
        'lot4_surface_carrez': 'float64',
        'lot5_numero': 'string',
        'lot5_surface_carrez': 'float64',
        'nombre_lots': 'float64',
        'code_type_local': 'string',
        'type_local': 'string',
        'surface_reelle_bati': 'float64',
        'nombre_pieces_principales': 'float64',
        'code_nature_culture': 'string',
        'nature_culture': 'string',
        'code_nature_culture_speciale': 'string',
        'nature_culture_speciale': 'string',
        'surface_terrain': 'float64',
        'longitude': 'float64',
        'latitude': 'float64'
    }

    # Conversion des types de colonnes
    for column_name, column_type in dict_type.items():
        if column_name in df.columns:  # Vérifie que la colonne existe
            try:
                df[column_name] = df[column_name].astype(column_type)
            except Exception as e:
                print(f"Erreur lors de la conversion de la colonne {column_name}: {e}")

    return df

def data_loader(path, departements=[], annees=[]):
    """
    Charge les fichiers CSV d'un répertoire et les concatène dans un DataFrame pandas.

    Parameters:
        path (str): Chemin vers le dossier contenant les données.
        departements (list): Liste des départements à inclure (ex: [1, 75]).
        annees (list): Liste des années à inclure (ex: [2020, 2021]).

    Returns:
        pd.DataFrame: Un DataFrame pandas contenant toutes les données chargées.
    """
    df = pd.DataFrame()

    # Liste des années
    annees_list = os.listdir(path) if not annees else [str(annee) for annee in annees]

    for annee in annees_list:
        cur_year = os.path.join(path, annee)

        # Liste des départements
        if not departements:
            departements_list = os.listdir(cur_year)
        else:
            departements_list = [f"{departement:02}.csv.gz" for departement in departements]

        for departement in departements_list:
            file = os.path.join(cur_year, departement)
            try:
                # Charger les données
                temp_df = pd.read_csv(file, low_memory=False)
                temp_df = change_column_types(temp_df)
                df = pd.concat([df, temp_df], ignore_index=True)
            except Exception as e:
                print(f"Erreur lors du chargement du fichier {file}: {e}")

    return df


In [ ]:
path = 'data_dvf'
df = data_loader(path,departements=[75,92,93,94,95,91,77,78])
df.head()

In [ ]:
df = df[
    df["valeur_fonciere"].notnull() &
    df["longitude"].notnull() &
    df["latitude"].notnull() &
    (df["surface_reelle_bati"].notna() | df["surface_terrain"].notna())
]

# Sélectionner les colonnes nécessaires
data = df[[
    'date_mutation', 'type_local', 'surface_reelle_bati', 'nombre_lots',
    'lot1_surface_carrez', 'lot2_surface_carrez', 'lot3_surface_carrez',
    'lot4_surface_carrez', 'lot5_surface_carrez', 'nombre_pieces_principales',
    'nature_culture', 'surface_terrain', 'longitude', 'latitude', 'valeur_fonciere'
]]

# Ajouter les colonnes calculées
data = data.assign(
    sin_month=np.sin(2 * np.pi * pd.to_datetime(data["date_mutation"]).dt.month / 12),
    cos_month=np.cos(2 * np.pi * pd.to_datetime(data["date_mutation"]).dt.month / 12),
    year=pd.to_datetime(data["date_mutation"]).dt.year,
    lot1_surface_carrez=data['lot1_surface_carrez'].fillna(0),
    lot2_surface_carrez=data['lot2_surface_carrez'].fillna(0),
    lot3_surface_carrez=data['lot3_surface_carrez'].fillna(0),
    lot4_surface_carrez=data['lot4_surface_carrez'].fillna(0),
    lot5_surface_carrez=data['lot5_surface_carrez'].fillna(0),
    surface_reelle_bati=data['surface_reelle_bati'].fillna(0),
    surface_terrain=data['surface_terrain'].fillna(0)
)
data.head()

In [ ]:
# Calculer la somme des surfaces Carrez pour créer la colonne "total_surface_carrez"
data["total_surface_carrez"] = (
    data["lot1_surface_carrez"] +
    data["lot2_surface_carrez"] +
    data["lot3_surface_carrez"] +
    data["lot4_surface_carrez"] +
    data["lot5_surface_carrez"]
)

# Si "total_surface_carrez" est 0, utiliser "surface_reelle_bati" à la place
data["total_surface_carrez"] = np.where(
    data["total_surface_carrez"] == 0,
    data["surface_reelle_bati"],
    data["total_surface_carrez"]
)

# Calculer "valeur_fonciere_m2" en fonction de "surface_reelle_bati" ou "surface_terrain"
data["valeur_fonciere_m2"] = np.where(
    data["surface_reelle_bati"] != 0,
    (data["valeur_fonciere"] / data["surface_reelle_bati"]).round(0),
    (data["valeur_fonciere"] / data["surface_terrain"]).round(0)
)
data = data[data['valeur_fonciere_m2'] != 0]
data["valeur_fonciere_m2_log"]= data["valeur_fonciere_m2"].apply(lambda x: np.log10(x))
data.head()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
x = data['valeur_fonciere_m2_log'].to_numpy()
# Calcul de la moyenne et de l'écart-type
mu, sigma = norm.fit(x)  # Ajuste une gaussienne et retourne la moyenne et l'écart-type

# Tracer l'histogramme des données
plt.hist(x, bins=100, density=True, alpha=0.6, color='skyblue', label="Données")

# Créer une courbe gaussienne avec les paramètres ajustés
x = np.linspace(min(x), max(x), 1000)
pdf = norm.pdf(x, mu, sigma)  # Fonction de densité de probabilité
plt.plot(x, pdf, color='red', linewidth=2, label=f"Fit Gaussien\n$\mu$={mu:.2f}, $\sigma$={sigma:.2f}")

# Ajouter des labels et une légende
plt.xlabel("Valeurs")
plt.ylabel("Densité")
plt.title("Ajustement Gaussien")
plt.legend()
plt.grid()
plt.show()

print(f"Moyenne (mu): {mu:.2f}")
print(f"Écart-type (sigma): {sigma:.2f}")


In [ ]:
from sklearn.ensemble import IsolationForest

model = IsolationForest(contamination=0.1)
data['anomalie'] = model.fit_predict(data[['valeur_fonciere_m2']])
df_cleaned = data[data['anomalie'] == 1]
print(f"{data.size-df_cleaned.size} lignes enlevées, taille finale : {df_cleaned.size}")


In [ ]:
x = df_cleaned['valeur_fonciere_m2'].to_numpy()
# Calcul de la moyenne et de l'écart-type
mu, sigma = norm.fit(x)  # Ajuste une gaussienne et retourne la moyenne et l'écart-type

# Tracer l'histogramme des données
plt.hist(x, bins=100, density=True, alpha=0.6, color='skyblue', label="Données")

# Créer une courbe gaussienne avec les paramètres ajustés
x = np.linspace(min(x), max(x), 1000)
pdf = norm.pdf(x, mu, sigma)  # Fonction de densité de probabilité
plt.plot(x, pdf, color='red', linewidth=2, label=f"Fit Gaussien\n$\mu$={mu:.2f}, $\sigma$={sigma:.2f}")

# Ajouter des labels et une légende
plt.xlabel("Valeurs")
plt.ylabel("Densité")
plt.title("Ajustement Gaussien")
plt.legend()
plt.grid()
plt.show()

print(f"Moyenne (mu): {mu:.2f}")
print(f"Écart-type (sigma): {sigma:.2f}")

In [ ]:
lower_surface = df_cleaned['surface_reelle_bati'].quantile(0.1)
upper_surface = df_cleaned['surface_reelle_bati'].quantile(0.95)
mask_surface = df_cleaned['surface_reelle_bati'].between(lower_surface, upper_surface)

lower_price_m2 = df_cleaned['valeur_fonciere_m2'].quantile(0.15)
upper_price_m2 = df_cleaned['valeur_fonciere_m2'].quantile(0.85)
mask_price_m2 = df_cleaned['valeur_fonciere_m2'].between(lower_price_m2, upper_price_m2)

df_cleaned[mask_surface & mask_price_m2].describe()


In [ ]:
df_cleaned = df_cleaned[mask_price_m2 & mask_surface]
df_cleaned.shape

In [12]:
from scipy.spatial import cKDTree

def idw_predict_kdtree(data, lon_col="longitude", lat_col="latitude", value_col="valeur_fonciere_m2", power=2, k=20):
    """
    Prédit les valeurs d'un DataFrame en utilisant l'Interpolation Inverse Distance Weighting (IDW) optimisée avec un KD-Tree.

    Paramètres :
        data : pd.DataFrame, DataFrame contenant les colonnes latitude, longitude et valeur foncière.
        lat_col : str, nom de la colonne latitude.
        lon_col : str, nom de la colonne longitude.
        value_col : str, nom de la colonne des valeurs à prédire.
        power : float, puissance de pondération (typiquement 2).
        k : int, nombre de voisins à considérer pour chaque point.

    Retourne :
        pd.Series, colonne des valeurs prédites pour chaque point.
    """
    # Extraire les coordonnées et les valeurs
    coordinates = data[[lat_col, lon_col]].to_numpy()
    values = data[value_col].to_numpy()

    # Construire le KD-Tree
    tree = cKDTree(coordinates,leafsize=36)

    predicted_values = []
    for i, point in enumerate(coordinates):
        # Trouver les k voisins les plus proches
        distances, indices = tree.query(point, k=k + 1)

        # Exclure le point lui-même (distance 0)
        mask = indices != i
        indices = indices[mask][:k]  # Prendre les k voisins valides

        # Si aucun voisin valide n'est trouvé
        if len(indices) == 0:
            predicted_values.append(values[i])  # Retourner la valeur du point lui-même
            continue

        # Calculer la valeur interpolée
        mean_value = np.mean(values[indices])
        predicted_values.append(mean_value)

    # Retourner les résultats sous forme de pd.Series
    return pd.Series(predicted_values, name=f"{value_col}_predite_par_le_quartier")


In [ ]:
# Appliquer la fonction idw_predict_kdtree et ajouter la colonne prédite au DataFrame pandas
value_col = "valeur_fonciere_m2"
df_cleaned[f"{value_col}_predite_par_le_quartier"] = idw_predict_kdtree(df_cleaned, lon_col="longitude", lat_col="latitude", value_col= value_col, k=36)
df_cleaned = df_cleaned[~df_cleaned['valeur_fonciere_m2_predite_par_le_quartier'].isna()]
df_cleaned = df_cleaned[np.isfinite(df_cleaned['valeur_fonciere_m2_predite_par_le_quartier'])]

# Afficher les premières lignes du DataFrame
df_cleaned.head()


In [ ]:
sns.scatterplot(data = df_cleaned[df_cleaned['valeur_fonciere_m2']< 15000], x = "valeur_fonciere_m2", y = "valeur_fonciere_m2_predite_par_le_quartier")

In [ ]:
import seaborn as sns

sns.histplot(data=df_cleaned,x="valeur_fonciere_m2")

In [ ]:
df_cleaned.describe()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
x = df_cleaned['valeur_fonciere_m2'].to_numpy()
# Calcul de la moyenne et de l'écart-type
mu, sigma = norm.fit(x)  # Ajuste une gaussienne et retourne la moyenne et l'écart-type

# Tracer l'histogramme des données
plt.hist(x, bins=100, density=True, alpha=0.6, color='skyblue', label="Données")

# Créer une courbe gaussienne avec les paramètres ajustés
x = np.linspace(min(x), max(x), 1000)
pdf = norm.pdf(x, mu, sigma)  # Fonction de densité de probabilité
plt.plot(x, pdf, color='red', linewidth=2, label=f"Fit Gaussien\n$\mu$={mu:.2f}, $\sigma$={sigma:.2f}")

# Ajouter des labels et une légende
plt.xlabel("Valeurs")
plt.ylabel("Densité")
plt.title("Ajustement Gaussien")
plt.legend()
plt.grid()
plt.show()

print(f"Moyenne (mu): {mu:.2f}")
print(f"Écart-type (sigma): {sigma:.2f}")


In [18]:
donnees = df_cleaned[(df_cleaned['valeur_fonciere_m2']<mu+3*sigma) & (df_cleaned['valeur_fonciere_m2']> mu-3*sigma)]

In [ ]:
df_popd = pd.read_csv('data_pop_density/dataframe_densite&amenities_radius=500.csv')
df_popd.drop(columns='Unnamed: 0',inplace=True)
df_popd

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import gamma

# Exemple : données positives
data_gamma = df_cleaned['valeur_fonciere_m2']

# Ajuster les paramètres de la distribution gamma
shape, loc, scale = gamma.fit(data_gamma, floc=0)  # floc=0 force le paramètre "loc" à zéro

print(f"Shape: {shape:.2f}, Scale: {scale:.2f}")

# Générer des valeurs pour tracer la densité de probabilité
x = np.linspace(min(data_gamma), max(data_gamma), 500)
pdf = gamma.pdf(x, shape, loc=loc, scale=scale)

# Tracer les données et la distribution ajustée
plt.figure(figsize=(10, 6))

# Histogramme des données
plt.hist(data_gamma, bins=50, density=True, alpha=0.6, color="skyblue", label="Données")

# Courbe de la densité ajustée
plt.plot(x, pdf, "red", label="Distribution Gamma ajustée")

# Ajouter des légendes et des titres
plt.title("Ajustement de la distribution Gamma", fontsize=14)
plt.xlabel("valeur_fonciere_m2", fontsize=12)
plt.ylabel("Densité", fontsize=12)
plt.legend(fontsize=12)
plt.grid(True)
plt.show()


In [21]:
import numpy as np
import pandas as pd
from scipy.spatial import cKDTree

def find_weighted_poi_counts(donnees, df_grid, poi_columns):
    """
    Calcule une moyenne pondérée des colonnes POIs des quatre voisins les plus proches
    pour chaque point du DataFrame `donnees` en utilisant un arbre k-d (cKDTree).

    Parameters:
    - donnees (pandas.DataFrame): DataFrame contenant les coordonnées des points pour lesquels
      les POIs pondérés doivent être calculés. Il doit contenir les colonnes 'latitude' et 'longitude'.
    - df_grid (pandas.DataFrame): DataFrame contenant le quadrillage avec coordonnées et colonnes POIs.
      Il doit contenir les colonnes 'lat', 'lon' et les colonnes spécifiées dans `poi_columns`.
    - poi_columns (list): Liste des noms des colonnes POIs dans `df_grid` à inclure dans les calculs.

    Returns:
    - pandas.DataFrame: DataFrame enrichi avec les colonnes pondérées des POIs.
    """
    # Extraire les coordonnées des deux ensembles
    latitudes_data = donnees['latitude'].values
    longitudes_data = donnees['longitude'].values

    latitudes_grid = df_grid['lat'].values
    longitudes_grid = df_grid['lon'].values

    # Créer un cKDTree pour une recherche rapide
    tree = cKDTree(np.vstack((longitudes_grid, latitudes_grid)).T)

    # Chercher les 4 voisins les plus proches pour chaque point
    distances, indices = tree.query(np.vstack((longitudes_data, latitudes_data)).T, k=4)

    # Calculer les poids en fonction de l'inverse des distances
    weights = 1 / np.where(distances == 0, 1e-10, distances)  # Évite la division par zéro
    normalized_weights = weights / weights.sum(axis=1, keepdims=True)

    # Calculer les moyennes pondérées pour chaque colonne POI
    for col in poi_columns:
        poi_values = df_grid[col].values
        # Récupérer les valeurs des voisins pour cette colonne
        neighbors_poi = poi_values[indices]
        # Calculer la moyenne pondérée
        donnees[f"{col}_weighted"] = np.floor((neighbors_poi * normalized_weights).sum(axis=1))

    return donnees


In [ ]:
poi_columns = ['densite','transport_pois', 'education_pois','health_pois', 'food_pois', 'shopping_pois', 'park_pois',	'entertainment_pois', 'cultural_pois']
donnees = find_weighted_poi_counts(donnees=donnees,df_grid=df_popd,poi_columns = poi_columns)
donnees.columns

# Pipeline

In [23]:
data_final = donnees.drop(['anomalie','valeur_fonciere_m2','valeur_fonciere_m2_log','valeur_fonciere_m2_predite_par_le_quartier'],axis = 1)
# data_final["valeur_fonciere_log"] = donnees['valeur_fonciere'].apply(lambda x: np.log10(x))
# Remplacer les valeurs <NA> par NaN
data_final['nature_culture'] = data_final['nature_culture'].replace(pd.NA, np.nan)
data_final['nature_culture'] = data_final['nature_culture'].astype(str)
data_final['type_local'] = data_final['type_local'].astype(str)

In [ ]:
data_final['date_mutation']

In [25]:
X = data_final.drop(['valeur_fonciere'],axis =1)
y= data_final['valeur_fonciere']
categorical_columns_onehot = ['nature_culture','type_local'] # Columns that need OneHotEncoding
numerical_columns = X.select_dtypes(include=['int32','float64']).columns


In [26]:
# Encoding and imputer Pipeline
unique_categories = [X[col].unique() for col in categorical_columns_onehot]

onehot_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', categories=unique_categories))
])

numeric_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(missing_values=np.nan,strategy='most_frequent')),  # Remplit les NaN avec 0
    ('scaler', MinMaxScaler())                 # Standardisation
])

# Encoding pipeline

column_transformer =  ColumnTransformer(
    transformers=[
        ('onehot', onehot_pipeline, categorical_columns_onehot),
        ('numeric', numeric_pipeline, numerical_columns)
    ]
)

In [ ]:
onehot_pipeline.fit(X[categorical_columns_onehot])

In [ ]:
numeric_pipeline.fit(X[numerical_columns])

In [ ]:
from sklearn import set_config

# Configuration pour afficher graphiquement
set_config(display="diagram")

# Afficher la pipeline dans un notebook
display(column_transformer)


In [30]:
def build_xgboost_model():
    pipeline = Pipeline(steps=[
        ('preprocessor', column_transformer),
        ('model', XGBRegressor(
            objective='reg:squarederror',  # Pour la régression
            n_estimators=300,             # Nombre d'arbres
            learning_rate=0.05,            # Taux d'apprentissage
            max_depth=10,                  # Profondeur maximale des arbres
            random_state=42               # Répétabilité
        ))
    ])
    return pipeline

In [ ]:
model = build_xgboost_model()
model

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
y_train

In [ ]:
model.fit(X_train,y_train)
y_pred = model.predict(X_test)
print("Mean Squared Error (MSE):", mean_squared_error(y_test, y_pred))
print("R2 Score:", r2_score(y_test, y_pred))

In [ ]:
sns.scatterplot(x=y_test,y=y_pred)

# Explicabilité 

In [ ]:
feature_names = model.named_steps['preprocessor'].get_feature_names_out()
# Supposons que ton modèle s'appelle `xgb_model`
xgb_model = model.named_steps['model']
# Récupérer l'importance des variables
importance = xgb_model.get_booster().get_score(importance_type='weight')

# Convertir en DataFrame
importance_df = pd.DataFrame.from_dict(importance, orient='index', columns=['weight']).reset_index()
importance_df.columns = ['feature', 'weight']

# Associer aux noms des colonnes transformées
importance_df.sort_values(by='weight', ascending=False)


In [ ]:
from xgboost import plot_importance
import matplotlib.pyplot as plt

plot_importance(xgb_model, importance_type="weight")
plt.show()

In [ ]:
import shap

# Crée un explainer SHAP
explainer = shap.TreeExplainer(xgb_model)

# Calcule les valeurs SHAP pour les données d'entraînement/test
shap_values = explainer.shap_values(X_test)

# Résumé global des contributions des variables
shap.summary_plot(shap_values, X_test)

# Visualiser l'impact d'une seule variable
shap.dependence_plot("nom_de_la_variable", shap_values, X_test)

# Interprétation individuelle pour une observation spécifique
shap.force_plot(explainer.expected_value, shap_values[0], X_test.iloc[0])


In [ ]:
from lime.lime_tabular import LimeTabularExplainer

# Crée un explainer LIME
explainer = LimeTabularExplainer(X_train.values, feature_names=X_train.columns, class_names=["Classe"], mode="regression")

# Explique une prédiction individuelle
exp = explainer.explain_instance(X_test.iloc[0].values, xgb_model.predict, num_features=10)

# Affiche les explications
exp.show_in_notebook(show_table=True)


In [ ]:
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense, Dropout
# from scikeras.wrappers import KerasRegressor
# from tensorflow.keras.optimizers import Adam
# from sklearn.pipeline import Pipeline
# from sklearn.compose import ColumnTransformer
# from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

# def build_deep_learning_model(input_shape):
#     model = Sequential([
#         Dense(512, activation='relu', input_shape=(input_shape,)),
#         Dense(256, activation='relu', input_shape=(input_shape,)),
#         Dense(128, activation='relu'),
#         Dense(64, activation='relu'),
#         Dense(128, activation='relu'),
#         Dropout(0.2),
#         Dense(256, activation='relu', input_shape=(input_shape,)),
#         Dropout(0.2),
#         Dense(512, activation='relu', input_shape=(input_shape,)),
#         Dropout(0.2),
#         Dense(1)  # Une seule sortie pour la régression
#     ])
#     model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['r2_score'])
#     return model

# def build_dl_pipeline(column_transformer, X_sample):
#     # Ajuster le ColumnTransformer pour obtenir les dimensions de sortie
#     column_transformer.fit(X_sample)
#     transformed_sample = column_transformer.transform(X_sample)

#     # Calculer la taille des features après transformation
#     input_shape = transformed_sample.shape[1]
#     print(f"Input shape for Keras model: {input_shape}")

#     early_stop = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)
#     reduce_lr = ReduceLROnPlateau(monitor='val_loss',
#                                   factor=0.2,    # Réduit le taux d'apprentissage par un facteur de 0.2
#                                   patience=5,    # Attendre 5 epochs sans amélioration avant de réduire le taux
#                                   min_lr=1e-6,   # Le taux d'apprentissage ne descendra pas en dessous de 1e-6
#                                   verbose=1)

#     keras_model = KerasRegressor(
#         build_fn=build_deep_learning_model,
#         input_shape=input_shape,  # Taille des features après prétraitement
#         epochs=200,
#         batch_size=16,
#         verbose=1,
#         callbacks = [early_stop,reduce_lr],
#         validation_split = 0.2
#     )

#     pipeline = Pipeline(steps=[
#         ('preprocessor', column_transformer),
#         ('model', keras_model)
#     ])
#     return pipeline


In [ ]:
# model_deep = build_dl_pipeline(column_transformer, X)

In [ ]:
# column_transformer.fit_transform(X_train).shape

In [36]:
# model_deep.fit(X_train,y_train)

In [37]:
# y_pred =model_deep.predict(X_test)
# print("Mean Squared Error (MSE):", mean_squared_error(y_test, y_pred))
# print("R2 Score:", r2_score(y_test, y_pred))

# plt.figure(figsize=(10, 6))
# sns.scatterplot(x=y_test, y=y_pred, alpha=0.7, label='Prédictions')
# sns.lineplot(x=y_test, y=y_test, color='red', label='Ligne parfaite (y_test = y_pred)')
# plt.xlabel("Valeurs réelles (y_test)")
# plt.ylabel("Valeurs prédites (y_pred)")
# plt.title("Comparaison des valeurs réelles et prédites")
# plt.legend()
# plt.show()

In [38]:
# # #best model
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense, Dropout
# from scikeras.wrappers import KerasRegressor
# from tensorflow.keras.optimizers import Adam
# from sklearn.pipeline import Pipeline
# from sklearn.compose import ColumnTransformer
# from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

# def build_deep_learning_model(input_shape):
#     model = Sequential([
#         Dense(512, activation='relu', input_shape=(input_shape,)),
#         Dense(256, activation='relu', input_shape=(input_shape,)),
#         Dense(128, activation='relu'),
#         Dense(64, activation='relu'),
#         Dense(128, activation='relu'),
#         Dropout(0.2),
#         Dense(256, activation='relu', input_shape=(input_shape,)),
#         Dropout(0.2),
#         Dense(512, activation='relu', input_shape=(input_shape,)),
#         Dropout(0.2),
#         Dense(1)  # Une seule sortie pour la régression
#     ])
#     model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['r2_score'])
#     return model

# def build_dl_pipeline(column_transformer, X_sample):
#     # Ajuster le ColumnTransformer pour obtenir les dimensions de sortie
#     column_transformer.fit(X_sample)
#     transformed_sample = column_transformer.transform(X_sample)

#     # Calculer la taille des features après transformation
#     input_shape = transformed_sample.shape[1]
#     print(f"Input shape for Keras model: {input_shape}")

#     early_stop = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)
#     reduce_lr = ReduceLROnPlateau(monitor='val_loss',
#                                   factor=0.2,    # Réduit le taux d'apprentissage par un facteur de 0.2
#                                   patience=5,    # Attendre 5 epochs sans amélioration avant de réduire le taux
#                                   min_lr=1e-6,   # Le taux d'apprentissage ne descendra pas en dessous de 1e-6
#                                   verbose=1)

#     keras_model = KerasRegressor(
#         build_fn=build_deep_learning_model,
#         input_shape=input_shape,  # Taille des features après prétraitement
#         epochs=200,
#         batch_size=16,
#         verbose=1,
#         callbacks = [early_stop,reduce_lr],
#         validation_split = 0.2
#     )

#     pipeline = Pipeline(steps=[
#         ('preprocessor', column_transformer),
#         ('model', keras_model)
#     ])
#     return pipeline
